In [20]:
# -*- coding: utf-8 -*-
#from flask_cors import CORS
#from flask import Flask, request, Response
import pandas as pd
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter,MarkdownTextSplitter
import langchain.text_splitter
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
# import IPython
import sentence_transformers
import json
from venus_api_base.venus_openapi import PyVenusOpenApi
import requests
from langchain.vectorstores import Chroma
import pandas as pd
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
#os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://omdbeta-ae-01.openai.azure.com"
os.environ["OPENAI_API_KEY"] = "***"



# app = Flask(__name__)
import urllib.request

# proxy_handler = urllib.request.ProxyHandler({'https': 'https://huggingface.co:443'})
# opener = urllib.request.build_opener(proxy_handler)
# urllib.request.install_opener(opener)

embedding_file = '***'

def create_app():
    app = Flask(__name__)
    CORS(app, supports_credentials=True)
    app.config.update(RESTFUL_JSON=dict(ensure_ascii=False))
    print("app创建ok")
    # db = data_init()
    # print("向量数据库ok")
    return app 

 

def data_init(db_init):
    # 加载文件夹
    embedding_file = '***'
    #db = Chroma(persist_directory=embedding_file, embedding_function=embeddings)
    qa_file = '***'

    ## 文档切分
    md_loader = TextLoader(qa_file, encoding='utf8')
    md_doc = md_loader.load()

    markdown_splitter = CharacterTextSplitter(separator='##',  chunk_size=380, chunk_overlap=0)
    md_docs = markdown_splitter.split_documents(md_doc)
    # print(len(md_docs))

    model_name = "BAAI/bge-base-zh-v1.5"  
    encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        encode_kwargs=encode_kwargs
    )

    embeddings.client = sentence_transformers.SentenceTransformer(embeddings.model_name)  
    if db_init == 1:
        print("开始初始化本地向量数据库") 
        db = Chroma.from_documents(md_docs, embeddings, persist_directory=embedding_file)
        print("首次初始化本地向量数据库-成功") 
    else :
        db = Chroma(persist_directory=embedding_file, embedding_function=embeddings)

        print("直接取持久化的向量数据库-成功")    
    return db



def RetriveQA(db, question, threshold):

    print("开始计算相似度")
    f_threshold = 0.35 + 0.0
    print(f"f_threshold:{f_threshold}")
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": f_threshold, "k": 3})
    similarDocs = retriever.get_relevant_documents(question)
    # print(f"similarDocs:{similarDocs}")

    docs_input = ''
    if len(similarDocs) > 0:
        docs_input = '原文参考段落：\n'
        for i in range(len(similarDocs)):
            docs_input = docs_input +    similarDocs[i].page_content + '\n'
            print("***********召回文本：",docs_input)
    else:
        print("针对问题：【",question,"】无召回文本!!!")
    return docs_input


def zongjie(answer, question ):
    # 生产
    print("调用Azure模型的api进行文档总结")
    prompt = f"你是一个问答小助手，请基于以下文档回答问题。文档：{answer} \
        请根据上述给定的文档，仔细思考，回答问题：{question} 要求回答时只能使用给定文档中直接相关的文本内容。忽略无关文本。回答要准确。\
        如果不能从文档中找出答案则回答：“抱歉，信息不足无法直接回答，请完善问题或参考以下文档。”，"
    
    # 对话类langchain API调用

    llm = AzureChatOpenAI(deployment_name="gpt-4")

    msg = HumanMessage(content=prompt)
    result = llm(messages=[msg])
    print(llm(messages=[msg]))
 
    return result

In [14]:
db = data_init(1)

Using embedded DuckDB with persistence: data will be stored in: /Users/lucianowang/Documents/Working/2023YEAR/202309_LLM/202311_乐问医学/embedding2


开始初始化本地向量数据库
首次初始化本地向量数据库-成功


In [16]:
embedding_file = '***'
model_name = "BAAI/bge-base-zh-v1.5"
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

question = '哪些人群是乳腺癌的高危人群'
threshold = 0.35
db = Chroma(persist_directory=embedding_file, embedding_function=embeddings)
docs_input = RetriveQA(db, question, threshold)
docs_input

Using embedded DuckDB with persistence: data will be stored in: /Users/lucianowang/Documents/Working/2023YEAR/202309_LLM/202311_乐问医学/embedding


开始计算相似度
f_threshold:0.35
***********召回文本： 原文参考段落：
（ 1 ）适合机会性筛查。##
（2 ）每 1~2 年进行 1 次乳腺 X 线摄影检查（极低证据级别 ，一般推荐）。##
第五节    乳腺癌高危人群筛查意见##
建议对乳腺癌高危人群提前进行筛查（小于 40 岁）， 筛查频度推荐每年 1 次 ，筛 查手段除了乳腺 X 线摄影检查之外 ，还可以采用 B 超 、MRI 等影像学手段（中证据级 别 ，一般推荐）。 一项在中国开展的随机对照研究比较了在高危人群中应用超声 、乳 腺 X 线摄影 ， 以及超声联合乳腺 X 线摄影作为 30～65 岁中国女性乳腺癌高危人群的 初筛手段 ，研究结果显示 ，超声作为初筛手段比乳腺 X 线摄影具有更高的灵敏度和相##
似的特异度（中证据级别）。##
1    罹患乳腺癌高危人群的定义##
存在下列情况之一者被认为是罹患乳腺癌的高危人群。

***********召回文本： 原文参考段落：
（ 1 ）适合机会性筛查。##
（2 ）每 1~2 年进行 1 次乳腺 X 线摄影检查（极低证据级别 ，一般推荐）。##
第五节    乳腺癌高危人群筛查意见##
建议对乳腺癌高危人群提前进行筛查（小于 40 岁）， 筛查频度推荐每年 1 次 ，筛 查手段除了乳腺 X 线摄影检查之外 ，还可以采用 B 超 、MRI 等影像学手段（中证据级 别 ，一般推荐）。 一项在中国开展的随机对照研究比较了在高危人群中应用超声 、乳 腺 X 线摄影 ， 以及超声联合乳腺 X 线摄影作为 30～65 岁中国女性乳腺癌高危人群的 初筛手段 ，研究结果显示 ，超声作为初筛手段比乳腺 X 线摄影具有更高的灵敏度和相##
似的特异度（中证据级别）。##
1    罹患乳腺癌高危人群的定义##
存在下列情况之一者被认为是罹患乳腺癌的高危人群。
（4 ）根据评估对象的年龄 、种族 、初潮年龄 、初产年龄 、个人乳腺疾病史 、乳 腺癌家族史和乳腺活检次数等多个风险因子 ，利用 Gail 模型进行罹患乳腺癌风险评##
估 。如果受试者 5 年内发病风险≥1.67% ，则被认为是高风险个体。##
注：一级亲属指母亲 、女儿和姐妹 ；二级亲属指姑 、姨 、祖母和外祖母。##
2    乳腺癌高危人群的筛查推荐策略与管理（低证据级别， 一般

'原文参考段落：\n（ 1 ）适合机会性筛查。##\n（2 ）每 1~2 年进行 1 次乳腺 X 线摄影检查（极低证据级别 ，一般推荐）。##\n第五节    乳腺癌高危人群筛查意见##\n建议对乳腺癌高危人群提前进行筛查（小于 40 岁）， 筛查频度推荐每年 1 次 ，筛 查手段除了乳腺 X 线摄影检查之外 ，还可以采用 B 超 、MRI 等影像学手段（中证据级 别 ，一般推荐）。 一项在中国开展的随机对照研究比较了在高危人群中应用超声 、乳 腺 X 线摄影 ， 以及超声联合乳腺 X 线摄影作为 30～65 岁中国女性乳腺癌高危人群的 初筛手段 ，研究结果显示 ，超声作为初筛手段比乳腺 X 线摄影具有更高的灵敏度和相##\n似的特异度（中证据级别）。##\n1    罹患乳腺癌高危人群的定义##\n存在下列情况之一者被认为是罹患乳腺癌的高危人群。\n（4 ）根据评估对象的年龄 、种族 、初潮年龄 、初产年龄 、个人乳腺疾病史 、乳 腺癌家族史和乳腺活检次数等多个风险因子 ，利用 Gail 模型进行罹患乳腺癌风险评##\n估 。如果受试者 5 年内发病风险≥1.67% ，则被认为是高风险个体。##\n注：一级亲属指母亲 、女儿和姐妹 ；二级亲属指姑 、姨 、祖母和外祖母。##\n2    乳腺癌高危人群的筛查推荐策略与管理（低证据级别， 一般推荐）##\n（ 1 ）推荐比起始年龄更早（<40 岁） 开展乳腺筛查。##\n（2 ）每年 1 次进行乳腺 X 线摄影检查。##\n（3 ）每 6~ 12 个月进行 1 次乳腺超声检查。##\n（4 ）每 6~ 12 个月进行 1 次乳腺体检。##\n（5 ）必要时联合乳腺增强 MRI 检查。##\n1  乳腺 X 线摄影检查技术规范##\n1.1  投照前准备工作\n中国几项大样本女性乳腺癌分子亚型研究结果显示 ， 中国女性乳腺癌病例中， Luminal A 型 占 40%~70% ， Luminal B 型 占 10%~20% ， 三 阴 型 乳 腺 癌 占 15%~20%，##\nHER2 阳性乳腺癌占 20%~30%。##\n第四节    乳腺癌高危因素##\n乳腺癌的病因和发病机制十分复杂 ， 是遗传因素 、生活方式和环境暴露等多  种因素及其相互作用的结果 。乳腺癌易感基因的遗传突变增加了乳腺癌的风险 ； 生殖因素

In [22]:
answer = docs_input
question = '哪些人群是乳腺癌的高危人群'
zongjie(answer, question)

调用Azure模型的api进行文档总结
content='乳腺癌的高危人群包括存在以下情况之一的个体：根据评估对象的年龄、种族、初潮年龄、初产年龄、个人乳腺疾病史、乳腺癌家族史和乳腺活检次数等多个风险因子，利用 Gail 模型进行罹患乳腺癌风险评估。如果受试者 5 年内发病风险≥1.67%，则被认为是高风险个体。'


AIMessage(content='乳腺癌的高危人群定义为存在以下情况之一的人群：根据评估对象的年龄、种族、初潮年龄、初产年龄、个人乳腺疾病史、乳腺癌家族史和乳腺活检次数等多个风险因子，利用 Gail 模型进行罹患乳腺癌风险评估。如果受试者 5 年内发病风险≥1.67%，则被认为是高风险个体。')